In Python, a generator allows for the creation of iterators without having to implement __iter__() and __next__() methods. Generators improve code readability, save memory by allowing for iterative access of elements, and allow for the traversal of infinite streams of data.

There are two types of generators in Python:

- Generator functions

- Generator Expressions

Both of these return a generator object that can be looped over similar to a list, but unlike a list, the contents of the generator object are not stored in memory, allowing for complex and even infinite iteration of data.

#### yield vs return

Generator functions are similar to regular functions except that they must return an iterator. But instead of using a return statement, generator functions use an expression called yield.

So how does yield differ from a return statement? Well, any code that is written after a yield expression will execute on the next iteration of the iterator. Code written after a return statement will not execute.
Another key difference between yield and return is that the yield expression will suspend the execution of the function and preserve any local variables that exist within the function. The return statement will terminate the function immediately and return the result(s) to the caller.

Like all objects, the iterator object returned by a generator function can be stored in a variable to be used later. It can then be iterated through as needed.

In [1]:
def class_standing_generator():
    yield 'Freshman'
    yield 'Sophomore'
    yield 'Junior'
    yield 'Senior'
class_standings = class_standing_generator()

for i in class_standings:
    print(i)

Freshman
Sophomore
Junior
Senior


#### next() and StopIteration

Generator functions return an iterator object that contains traversable values. To retrieve the next value from a generator object, we can use the Python built-in function next() which will cause the generator function to resume its execution until the next yield expression is found. After the next yield expression is found, the function will pause execution again.

If no additional yield expressions are found in a generator function, that means the code has finished and a StopIteration is raised.

Generator functions are not limited to just single yield statements. They can also include loops where the yield occurs.

In [3]:
def student_standing_generator():
    student_standings = ['Freshman','Senior', 'Junior', 'Freshman']
    for st in student_standings:
        if st == 'Freshman':
            yield 500

standing_values = student_standing_generator()

print(next(standing_values))
print(next(standing_values))
print(next(standing_values))

500
500


StopIteration: 

#### Generator Expressions

In [6]:
def cs_generator():
    for i in range(1,5):
        yield "Computer Science " + str(i)

cs_courses = cs_generator()

for i in cs_courses:
    print(i)

Computer Science 1
Computer Science 2
Computer Science 3
Computer Science 4


In [5]:
cs_generator_exp = ("Computer Science " + str(i) for i in range(1, 5))
for i in cs_generator_exp:
    print(i)

Computer Science 1
Computer Science 2
Computer Science 3
Computer Science 4


In [7]:
a_list = [i*i for i in range(4)]
print(a_list)

[0, 1, 4, 9]


#### Generator Methods: send()

The .send() method allows us to send a value to a generator using the yield expression. If you assign yield to a variable the argument passed to the .send() method will be assigned to that variable. Calling .send() will also cause the generator to perform an iteration.

In [14]:
def generator():
    count = 0
    while True:
        n = yield count
        if n is not None:
            count = n
        count += 1

my_generator = generator()

print(next(my_generator)) 
print(next(my_generator)) 
print(my_generator.send(6)) 
print(next(my_generator)) 
print(my_generator.send(2)) 
print(next(my_generator)) 
print(next(my_generator)) 
print(my_generator.send(9)) 
print(next(my_generator)) 
print(next(my_generator)) 

0
1
7
8
3
4
5
10
11
12


In [2]:
MAX_STUDENTS = 50

def get_student_ids():
    student_id = 1
    while student_id <= MAX_STUDENTS:
      
        n = yield student_id
        if n is not None:
            student_id = n 
            continue
        student_id += 1

student_id_generator = get_student_ids()
for i in student_id_generator:
    if i == 1:
        i = student_id_generator.send(45)
    print(i)

45
46
47
48
49
50


#### Generator Methods: throw()

The generator method throw() provides the ability to throw an exception inside the generator from the caller point. This can be useful if we need to end the generator once it reaches a certain value or meets a particular condition.

In [6]:
def student_counter():
    for i in range(1,100):
        yield i

student_generator = student_counter()
for student_id in student_generator:
    if student_id > 10:
        student_generator.throw(ValueError, "Invalid student ID")
  
    print(student_id)

1
2
3
4
5
6
7
8
9
10


ValueError: Invalid student ID

#### Generator Methods: close()

The generator method .close() is used to terminate a generator early. Once the .close() method is called the generator is finished just like the end of a for loop. Any further iteration attempts will raise a StopIteration exception.

In [7]:
def generator():
    i = 0
    while True:
        try:
            yield i
        except GeneratorExit:
            print("Early exit, BYE!")
            break
        i += 1

my_generator = generator()
for item in my_generator:
    print(item)
    if item == 1:
        my_generator.close()

0
1
Early exit, BYE!


#### Connecting Generators

There are some cases where it is useful to connect multiple generators into one. This allows us to delegate the operations of one generator to another sub-generator. Connecting generators is similar to using the itertools chain() function to combine iterators into a single iterator.

In [9]:
def science_students(x):
    for i in range(1,x+1):
        yield i

def non_science_students(x,y):
    for i in range(x,y+1):
        yield i
        
def combined_students():
    yield from science_students(5)
    yield from non_science_students(10,15)
    yield from non_science_students(25,30)

student_generator = combined_students()

for i in student_generator:
    print(i)

1
2
3
4
5
10
11
12
13
14
15
25
26
27
28
29
30


#### Generator Pipelines

Generator pipelines allow us to use multiple generators to perform a series of operations all within one expression. We can break down complex operations into smaller, more manageable parts where they can then be pipelined together to achieve the desired output.

To pipeline generators, the output of one generator function can be the input of another generator function. That resulting generator can then be used as input for another generator function, and so on.

Pipeline generators are also often referred to as nested generators. 

In [11]:
def course_generator():
    yield ('Computer Science', 5)
    yield ('Art', 10)
    yield ('Business', 15)

def add_five_students(courses):
    for course, num_students in courses:
        yield (course, num_students + 5)

increased_courses = add_five_students(course_generator())
for course in increased_courses:
    print(course)
    

('Computer Science', 10)
('Art', 15)
('Business', 20)


In [12]:
def summa():
    yield 'Summa Cum Laude'

def magna():
    yield 'Magna Cum Laude' 

def cum_laude():
    yield 'Cum Laude'

def honors_generator(gpas):
    for gpa in gpas:
        if gpa > 3.9:
            yield from summa()
        elif gpa > 3.7:
            yield from magna()
        elif gpa > 3.5:
            yield from cum_laude()


def graduation_countdown(days):
    while days >= 0:
        days_left = yield days
        if days_left != None:
            days = days_left
        else:
            days -= 1


days = 25
countdown_generator = (day for day in range(days, -1,-1))
grad_days = graduation_countdown(days)
for day in grad_days:
    if day == 15:
        grad_days.send(10)
    elif day == 3:
        grad_days.close()
    print("Days Left: " + str(day))


days = 25
gpas = [3.2, 4.0, 3.6, 2.9]
honors = honors_generator(gpas)
for honor_label in honors:
    print(honor_label)

Days Left: 25
Days Left: 24
Days Left: 23
Days Left: 22
Days Left: 21
Days Left: 20
Days Left: 19
Days Left: 18
Days Left: 17
Days Left: 16
Days Left: 15
Days Left: 9
Days Left: 8
Days Left: 7
Days Left: 6
Days Left: 5
Days Left: 4
Days Left: 3
Summa Cum Laude
Cum Laude
